# Keras Tuner: Reuters Dataset

## Conjunto de datos

Este es un conjunto de datos de 11.228 noticias de Reuters, etiquetadas en 46 temas.

Esto fue originalmente generado por el análisis y preprocesamiento del clásico conjunto de datos de Reuters-21578, pero el código de preprocesamiento ya no está empaquetado con Keras. 

Cada noticia está codificado como una lista de índices de palabras (números enteros). Para mayor comodidad, las palabras se indexan por la frecuencia general en el conjunto de datos, de modo que, por ejemplo, el número entero "3" codifica la tercera palabra más frecuente en los datos. Esto permite realizar operaciones de filtrado rápido como: "considerar sólo las 10.000 palabras más comunes, pero eliminar las 20 palabras más comunes".

## 1. Cargando el conjunto de datos

In [ ]:
from tensorflow.keras.datasets import reuters

In [ ]:
(train_data, train_labels), (test_data, test_labels) = reuters.load_data(num_words=10000)

## Preprocesamiento del conjunto de datos

A pesar de que ya tenemos codificadas las reseñas para que esten representadas por valores numéricos en lugar de por cadenas de texto, existen algunos factores que no permiten que sean conjuntos de datos apropiados para proporcionarle a una red neuronal artifical.

In [ ]:
import numpy as np
def vectorize(seqs, dim=10000):
    results = np.zeros((len(seqs), dim))
    for i, seq in enumerate(seqs):
        results[i, seq] = 1.
    return results

Vectorizamos tanto el conjunto de datos de entrenamiento, como el conjunto de datos de pruebas.

In [ ]:
# Vectorizamos el conjunto de datos de entrenamiento y el de pruebas
X_train = vectorize(train_data, 10000)
X_test = vectorize(test_data, 10000)

In [ ]:
# Cada una de las reseñas se corresponde con un vector formado por 1s y 0s
print("Valores originales:\t", train_data[0][:15])
print("Valores vectorizados:\t", X_train[0][:15])

In [ ]:
# Transformamos las etiquetas de salida en vectores
from tensorflow.keras.utils import to_categorical
Y_train = to_categorical(train_labels)
Y_test = to_categorical(test_labels)

## División del conjunto de datos 

In [ ]:
from sklearn.model_selection import train_test_split
X_test, X_val, Y_test, Y_val = train_test_split(X_test, Y_test, test_size=0.5)

In [ ]:
print("Longitud subconjunto de entrenamiento: ", len(X_train))
print("Longitud subconjunto de validación: ", len(X_val))
print("Longitud subconjunto de pruebas: ", len(X_test))

## Construcción de la red neuronal artificial

### Función para la realización del tuning

In [ ]:
from tensorflow import keras
from tensorflow.keras import models
from tensorflow.keras import layers
import kerastuner as kt

In [ ]:
def model_builder(hp):
    # Definición del modelo
    model = keras.Sequential()
    # Tuning del número de neuronas de TODAS las hidden layer
    hp_units = hp.Int('units', min_value=8, max_value=128, step=8)
    # Tuning del número de capas para cada capa hidden
    hp_layers = hp.Int('layers', min_value=1, max_value=5, step=1)
    # Input layer
    model.add(layers.Dense(units=128, activation='relu', input_shape=(10000,)))
    # Hidden layers 
    for i in range(hp_layers):
        # Añadimos una hidden layer en cada iteración del bucle
        model.add(layers.Dense(units=hp_units, activation='relu'))
    # Output layer -> 46 temas posibles en las noticias
    model.add(layers.Dense(46, activation='softmax'))
    # Seleccionamos el valor optimo entre [0.01, 0.001, 0.0001]
    hp_learning_rate = hp.Choice('learning_rate', values=[1e-2, 1e-3, 1e-4])
    model.compile(
        optimizer=keras.optimizers.Adam(learning_rate=hp_learning_rate), 
        loss='categorical_crossentropy',
        metrics=['accuracy']
    )
    return model

In [ ]:
tuner = kt.Hyperband(
    model_builder,
    objective= 'val_accuracy',
    max_epochs=10,
    factor=3, # nº de entrenamientos por ronda del Hyper
    directory='hp_dir',
    project_name='ruters_dataset'
)

In [ ]:
# Creamos una función de callback que limpie el output tras entrenar cada modelo del Hypertunning en el .search()
import IPython
import tensorflow as tf
class ClearTrainingOutput(tf.keras.callbacks.Callback):
    def on_train_end(*args, **kwargs):
        IPython.display.clear_output(wait = True)

In [ ]:
# Ejecución del proceso de tuning y al final ejecuta el callback
tuner.search(
    X_train, 
    Y_train,
    epochs=10,
    validation_data=(X_val, Y_val),
    callbacks=[ClearTrainingOutput()]
)

In [ ]:
# Obtención de los hiperparámetros óptimos -> el best_hps es el modelo que ha encontrado como óptimo
best_hps = tuner.get_best_hyperparameters(num_trials=1)[0]
print("Units:", best_hps.get('units'))
print("Layers:", best_hps.get('layers'))

In [ ]:
model = tuner.hypermodel.build(best_hps)
history = model.fit(
    X_train,
    Y_train,
    epochs = 20,
    validation_data = (X_val, Y_val)
)

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
pd.DataFrame(history.history)[['loss', 'val_loss']].plot(figsize=(10, 6))
plt.grid(True)
plt.xlabel("epochs")
plt.ylabel("loss")
plt.show()

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
pd.DataFrame(history.history)[['accuracy', 'val_accuracy']].plot(figsize=(10, 6))
plt.grid(True)
plt.xlabel("epochs")
plt.ylabel("accuracy")
plt.show()

In [ ]:
model.evaluate(X_test, Y_test)

In [ ]:
model.predict(X_test).round(0)